# Generation of a quantum dataset

In [220]:
import numpy as np
import torch as th
import os
from scipy.linalg import eigvals

In [221]:
def gram_schmidt(vectors):
    # Convert the input list of vectors to a NumPy array
    vectors = np.array(vectors, dtype=complex)
    
    # Get the number of vectors and their dimensionality
    num_vectors, vector_dim = vectors.shape
    
    # Create an array to store the orthogonalized vectors
    orthogonalized_vectors = np.zeros((num_vectors, vector_dim), dtype=complex)
    
    for i in range(num_vectors):
        # Orthogonalize the current vector with respect to the previous ones
        orthogonalized_vectors[i] = vectors[i]
        for j in range(i):
            orthogonalized_vectors[i] -= np.dot(vectors[i], orthogonalized_vectors[j]) / np.dot(orthogonalized_vectors[j], orthogonalized_vectors[j]) * orthogonalized_vectors[j]
        
        # Normalize the orthogonalized vector
        orthogonalized_vectors[i] /= np.linalg.norm(orthogonalized_vectors[i])
    
    return orthogonalized_vectors

def generate_orthogonal_vectors(n, dim):
    # Generate N random vectors
    random_vectors = [np.random.rand(dim) for _ in range(n)]

    # Gram-Schmidt orthogonalization
    orthogonalized_vectors = gram_schmidt(random_vectors)

    return orthogonalized_vectors


# def generate_orthogonal_vectors(n, dim):
#     # Generate N random vectors
#     random_vectors = [np.random.rand(dim) + 1j * np.random.rand(dim) for _ in range(n)]

#     # Gram-Schmidt orthogonalization
#     orthogonalized_vectors = gram_schmidt(random_vectors)

#     return orthogonalized_vectors

In [222]:
def generate_density_matrices(orthogonal_vectors):
    N = len(orthogonal_vectors)
    density_matrix = []
    for i in range(N):
        density_matrix.append(np.outer(orthogonal_vectors[i], orthogonal_vectors[i]))
        
    # pose trace = 1
    return np.array(density_matrix)

In [223]:
def generate_coefficients(N):
    coefficients = np.random.rand(N)
    coefficients /= np.sum(coefficients)
    return coefficients

In [224]:
def generate_separable_states(n_states, N, dim):
    """Generate a separable state of 2 subsystems with N basis, each of dimension dim.
    output: a list of n_states matrices of dimension N**2 x N**2"""
    
    states = []
    for i in range(n_states):
        orthogonal_vectorsA = generate_orthogonal_vectors(N, dim)
        orthogonal_vectorsB = generate_orthogonal_vectors(N, dim)
        
        density_matrixA = generate_density_matrices(orthogonal_vectorsA)
        density_matrixB = generate_density_matrices(orthogonal_vectorsB)
        coefficients = generate_coefficients(N)
        result = 0
        for i in range(N):
            result += coefficients[i] * np.kron(density_matrixA[i], density_matrixB[i])
        states.append(result)
    
    return np.array(states)

In [228]:
def is_entangled(rho):
    # Check if the density matrix is 4x4
    if rho.shape != (4, 4):
        raise ValueError("The input matrix should be a 4x4 density matrix for a two-qubit system.")
    
    # Calculate the partial transpose of the density matrix
    rho_T_B = np.kron(np.eye(2), np.array([[1, 0], [0, -1]])) @ np.conjugate(np.transpose(rho)) @ np.kron(np.eye(2), np.array([[1, 0], [0, -1]]))
    
    # Check if any eigenvalue of the partial transpose is negative
    eigenvalues = eigvals(rho_T_B)
    
    return any(eig < 0 for eig in eigenvalues)

In [237]:
sep_states = generate_separable_states(n_states = 10, N = 2, dim = 2)

for i in range(len(sep_states)):
    print(any(eigvals(sep_states[i]) < 0), is_entangled(sep_states[i]))

False False
True True
True True
True True
False False
True True
True True
True True
False False
False False


In [214]:
def generate_separable_state(probabilities, states_A, states_B):
    # Check if the input is valid
    if len(probabilities) != len(states_A) or len(probabilities) != len(states_B):
        raise ValueError("Input lists should have the same length.")
    if not np.isclose(np.sum(probabilities), 1.0):
        raise ValueError("Probabilities should sum to 1.")

    # Number of terms in the mixture
    num_terms = len(probabilities)

    # Generate the separable state
    separable_state = np.zeros((4, 4), dtype=complex)
    for i in range(num_terms):
        separable_state += probabilities[i] * np.kron(states_A[i], states_B[i])

    return separable_state

# Example usage
# Define probabilities and dimensionality
probabilities = [0.3, 0.7]
state_A_1 = np.array([[1, 0], [0, 0]])
state_A_2 = np.array([[0, 0], [0, 1]])
state_B_1 = np.array([[1, 0], [0, 0]])
state_B_2 = np.array([[0, 0], [0, 1]])

# Generate the separable state
separable_state = generate_separable_state(probabilities, [state_A_1, state_A_2], [state_B_1, state_B_2])


print("Separable State:")
print(separable_state)

is_entangled(separable_state)


Separable State:
[[0.3+0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0.7+0.j]]
[0.3+0.j 0. +0.j 0. +0.j 0.7+0.j]


False

In [211]:
for i in range(10):
    print(is_entangled(sep_states[i]))

[-7.96846793e-01-4.86079875e-01j  6.53675262e-02-1.24939630e-03j
  6.97421253e-18-8.82956992e-18j -3.65477402e-17+1.05709020e-17j]
True
[ 1.96995658e-01+4.60100714e-01j  2.78720608e-17-4.89754550e-18j
 -1.03288643e-01+1.83457074e-01j -3.41760262e-17-2.62977488e-17j]
True
[-2.94434923e-01+2.46392074e-01j  3.37061608e-01-4.95069278e-02j
 -2.23537955e-16-9.09810252e-17j -1.97743868e-17+7.85839519e-17j]
True
[-7.50052261e-01+6.09030867e-01j -1.51968499e-02-2.63197147e-02j
  4.84299658e-19-1.82266055e-17j -2.58847238e-17+3.87976027e-18j]
True
[-3.80664876e-01+2.33054691e-01j -6.04200477e-02-3.24472009e-01j
  5.38380981e-17+1.54594865e-18j  1.24476001e-18+1.08143464e-17j]
True
[-4.23863765e-01+4.83969195e-01j -2.51075553e-01+8.05267314e-02j
  2.12287625e-17-3.52586729e-17j  5.29674108e-17-6.87076116e-17j]
True
[ 2.76105658e-01-3.52391996e-01j -3.74880566e-01-5.93311698e-03j
 -2.61030163e-17-1.15001657e-16j -1.09018980e-17+2.96140037e-17j]
True
[ 4.19340869e-01-3.02194262e-01j  2.24926002e-17

In [239]:

def generate_hermitian_product_states(size, n_matrices):
    """Generates a list of random Hermitian product states
    of the given dimension
    Product states are hermitian matrices with trace 1.
    Input:
        size: size of the matrices
        n_matrices: number of matrices to generate
    Output:
        product_states: 3D numpy array of product states
    """

    product_states = []
    for _ in range(n_matrices):
        real_part = np.random.rand(size, size)
        imag_part = np.random.rand(size, size)
        product_state = real_part + 1j * imag_part
        product_state = np.matmul(product_state, product_state.conj().T)
        product_state /= np.trace(product_state)
        product_states.append(product_state)

    return np.array(product_states)

In [313]:
rhoA = generate_hermitian_product_states(2, 10)
rhoB = generate_hermitian_product_states(2, 10)

coeffs = np.random.rand(10)

sep_state = 0
partial_transpose = 0

for i in range(10):
    sep_state += coeffs[i] * np.kron(rhoA[i], rhoB[i])
    partial_transpose += coeffs[i] * np.kron(rhoA[i], np.transpose(rhoB[i]))
    
    
is_entangled(sep_state)

False

In [314]:
sep_state

array([[1.09048825+0.j        , 0.6940669 +0.25648125j,
        0.85674978-0.24909034j, 0.66221495-0.04080878j],
       [0.6940669 -0.25648125j, 0.81636177+0.j        ,
        0.40634123-0.33257812j, 0.56688773-0.30154527j],
       [0.85674978+0.24909034j, 0.40634123+0.33257812j,
        1.27085409+0.j        , 0.8360074 +0.1841741j ],
       [0.66221495+0.04080878j, 0.56688773+0.30154527j,
        0.8360074 -0.1841741j , 0.87439025+0.j        ]])

In [315]:
partial_transpose

array([[1.09048825+0.j        , 0.6940669 -0.25648125j,
        0.85674978-0.24909034j, 0.40634123-0.33257812j],
       [0.6940669 +0.25648125j, 0.81636177+0.j        ,
        0.66221495-0.04080878j, 0.56688773-0.30154527j],
       [0.85674978+0.24909034j, 0.66221495+0.04080878j,
        1.27085409+0.j        , 0.8360074 -0.1841741j ],
       [0.40634123+0.33257812j, 0.56688773+0.30154527j,
        0.8360074 +0.1841741j , 0.87439025+0.j        ]])

In [316]:
def calculate_partial_transpose(rho):
    # Check if the density matrix is 4x4
    if rho.shape != (4, 4):
        raise ValueError("The input matrix should be a 4x4 density matrix for a two-qubit system.")
    
    # Calculate the partial transpose of the density matrix
    rho_ur = np.transpose(rho[0:2, 2:4])
    rho_ll = np.transpose(rho[2:4, 0:2])

    rho, rho_ur, rho_ll

    rho_TB_u = np.concatenate([rho[0:2, 0:2], rho_ur], axis = 1)
    rho_TB_l = np.concatenate([rho_ll, rho[2:4, 2:4]], axis = 1)

    rho_T_B = np.concatenate([rho_TB_u, rho_TB_l], axis = 0)
    
    return rho_T_B

In [317]:
rho_tb = calculate_partial_transpose(sep_state)

rho_tb

array([[1.09048825+0.j        , 0.6940669 +0.25648125j,
        0.85674978-0.24909034j, 0.40634123-0.33257812j],
       [0.6940669 -0.25648125j, 0.81636177+0.j        ,
        0.66221495-0.04080878j, 0.56688773-0.30154527j],
       [0.85674978+0.24909034j, 0.66221495+0.04080878j,
        1.27085409+0.j        , 0.8360074 +0.1841741j ],
       [0.40634123+0.33257812j, 0.56688773+0.30154527j,
        0.8360074 -0.1841741j , 0.87439025+0.j        ]])

In [321]:
mat1 = np.array([[1, 0], [0, -1]])
identity_mat = np.eye(2)
transpose_B = np.kron(identity_mat, mat1)
trans_mat = np.transpose(sep_state)
rho_T_B = transpose_B @ trans_mat @ transpose_B

rho_T_B

array([[ 1.09048825+0.j        , -0.6940669 +0.25648125j,
         0.85674978+0.24909034j, -0.66221495-0.04080878j],
       [-0.6940669 -0.25648125j,  0.81636177+0.j        ,
        -0.40634123-0.33257812j,  0.56688773+0.30154527j],
       [ 0.85674978-0.24909034j, -0.40634123+0.33257812j,
         1.27085409+0.j        , -0.8360074 +0.1841741j ],
       [-0.66221495+0.04080878j,  0.56688773-0.30154527j,
        -0.8360074 -0.1841741j ,  0.87439025+0.j        ]])

In [324]:
rho = np.array(([1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11 ,12], [13, 14, 15, 16]))

rho, transpose_B @ rho.T @ transpose_B

(array([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]]),
 array([[  1.,  -5.,   9., -13.],
        [ -2.,   6., -10.,  14.],
        [  3.,  -7.,  11., -15.],
        [ -4.,   8., -12.,  16.]]))

In [319]:
rand_state = np.random.rand(4, 4)

is_entangled(rand_state)

# sep_state = 0

# for i in range(10):
#     sep_state += coeffs[i] * np.kron(rhoA[i], rhoB[i])
    
# is_entangled(sep_state)

False